# Predicting Credit Card Fraud

Credit card fraud is one of the leading causes of identify theft around the world. In 2018 alone, over $24 billion were stolen through fraudulent credit card transactions. Financial institutions employ a wide variety of different techniques to prevent fraud, one of the most common being Logistic Regression.

In this project, you are a Data Scientist working for a credit card company. You have access to a dataset (based on a synthetic financial dataset), that represents a typical set of credit card transactions. Your task is to use Logistic Regression and create a predictive model to determine if a transaction is fraudulent or not.

In [25]:
import seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [26]:
# Load the data
transactions = pd.read_csv('transactions.csv')
transactions.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,8,CASH_OUT,158007.12,C424875646,0.00,0.00,C1298177219,474016.32,1618631.97,0
1,236,CASH_OUT,457948.30,C1342616552,0.00,0.00,C1323169990,2720411.37,3178359.67,0
2,37,CASH_IN,153602.99,C900876541,11160428.67,11314031.67,C608741097,3274930.56,3121327.56,0
3,331,CASH_OUT,49555.14,C177696810,10865.00,0.00,C462716348,0.00,49555.14,0
4,250,CASH_OUT,29648.02,C788941490,0.00,0.00,C1971700992,56933.09,86581.10,0


In [27]:
transactions.count()

step              199999
type              199999
amount            199999
nameOrig          199999
oldbalanceOrg     199999
newbalanceOrig    199999
nameDest          199999
oldbalanceDest    199999
newbalanceDest    199999
isFraud           199999
dtype: int64

In [28]:
# Summary statistics on amount column
transactions['amount'].describe()

count    1.999990e+05
mean     1.802425e+05
std      6.255482e+05
min      0.000000e+00
25%      1.338746e+04
50%      7.426695e+04
75%      2.086376e+05
max      5.204280e+07
Name: amount, dtype: float64

We have a lot of information about the type of transaction we are looking at. Let’s create a new column called isPayment that assigns a 1 when type is “PAYMENT” or “DEBIT”, and a 0 otherwise.

In [29]:
# Create isPayment field
transactions['isPayment'] = 0
transactions['isPayment'][transactions['type'].isin(['PAYMENT','DEBIT'])] = 1

/var/folders/pt/vndzxn3j6qgfjm0yndf85y8w0000gn/T/ipykernel_24022/932289185.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions['isPayment'][transactions['type'].isin(['PAYMENT','DEBIT'])] = 1


Similarly, create a column called isMovement, which will capture if money moved out of the origin account. This column will have a value of 1 when type is either “CASH_OUT” or “TRANSFER”, and a 0 otherwise.

In [30]:
# Create isMovement field
transactions['isMovement'] = 0
transactions['isMovement'][transactions['type'].isin(['CASH_OUT', 'TRANSFER'])] = 1

/var/folders/pt/vndzxn3j6qgfjm0yndf85y8w0000gn/T/ipykernel_24022/3197595553.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions['isMovement'][transactions['type'].isin(['CASH_OUT', 'TRANSFER'])] = 1


With financial fraud, another key factor to investigate would be the difference in value between the origin and destination account. Our theory, in this case, being that destination accounts with a significantly different value could be suspect of fraud. Let’s create a column called accountDiff with the absolute difference of the oldbalanceOrg and oldbalanceDest columns.

In [31]:
# Create accountDiff field
transactions['accountDiff'] = abs(transactions['oldbalanceDest'] - transactions['oldbalanceOrg'])

In [32]:
# Create features and label variables
features = transactions[['amount','isPayment','isMovement','accountDiff']]
label = transactions['isFraud']

In [33]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.3)

In [34]:
# Normalize the features variables
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [35]:
# Fit the model to the training data
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [36]:
# Score the model on the training data
print(model.score(X_train, y_train))

0.998571418367274


In [37]:
# Score the model on the test data
print(model.score(X_test, y_test))

0.99855


In [38]:
# Print the model coefficients
print(model.coef_)

[[ 0.21524652 -0.72890537  2.26993454 -0.5565564 ]]


In [39]:
# New transaction data
transaction1 = np.array([123456.78, 0.0, 1.0, 54670.1])
transaction2 = np.array([98765.43, 1.0, 0.0, 8524.75])
transaction3 = np.array([543678.31, 1.0, 0.0, 510025.5])

In [40]:
# Create a new transaction
your_transaction = np.array([6472.54, 1.0, 0.0, 55901.23])

In [41]:
# Combine new transactions into a single array
sample_transactions = np.stack((transaction1,transaction2,transaction3,your_transaction))

In [42]:
# Normalize the new transactions
sample_transactions = scaler.transform(sample_transactions)

In [43]:
# Predict fraud on the new transactions
print(model.predict(sample_transactions))

[0 0 0 0]


In [44]:
# Show probabilities on the new transactions
print(model.predict_proba(sample_transactions))

[[9.96631002e-01 3.36899842e-03]
 [9.99992526e-01 7.47413408e-06]
 [9.99991890e-01 8.10952380e-06]
 [9.99992807e-01 7.19331231e-06]]
